### В данном файле представлено обучение разных моделей, подбор гиперпараметров, стекинг

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy

from dataclasses import dataclass, field  
from typing import List, Dict, Any  
from abc import ABC, abstractmethod

from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier, DMatrix

from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

import optuna
optuna.logging.set_verbosity(30)

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import recall_score

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer, KNNImputer

import warnings
import warnings; warnings.filterwarnings("ignore")

### Загрузим используемый датасет

In [2]:
train_data = pd.read_pickle('train')

drop_features = ['car_id']
targets = ['target_class', 'target_reg']
categorical_features = ['model', 'car_type', 'fuel_type']
filtered_features = [i for i in train_data.columns if (i not in targets and i not in drop_features)]
numerical_features = [i for i in filtered_features if i not in categorical_features and i not in drop_features]   
    
X = train_data[filtered_features]
y = train_data['target_class']

# Переведем метки классов в числа
le_tar = LabelEncoder()
y = pd.Series(le_tar.fit_transform(y)) 

# Изменим типы признаков
for cat in categorical_features:
    X[cat] = X[cat].astype('category')

display(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_mean,user_uniq,mean_destroy_degree,std_destroy_degree,absolute_difference,relative_difference
0,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,4.737759,1.214131e+07,0.10,180.855726,0.023174,-0.120391,170,3.048571,2.732847,9.001,-4.5005
1,VW Polo VI,economy,petrol,3.90,2015,78218,2021,4.480517,1.803909e+07,0.00,187.862734,12.306011,6.050011,174,2.917143,2.707233,72.761,-36.3805
2,Renault Sandero,standart,petrol,6.30,2012,23340,2017,4.768391,1.588366e+07,0.10,102.382857,2.513319,-2.223954,173,3.740000,2.978077,16.401,-8.2005
3,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,3.880920,1.651883e+07,0.10,172.793237,-5.029476,14.771948,170,4.085714,3.237750,61.647,-30.8235
4,Renault Sandero,standart,petrol,4.70,2012,26428,2017,4.181149,1.398317e+07,0.10,203.462289,-14.260456,12.455678,171,3.880000,3.216758,70.176,-35.0880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332,Smart ForFour,economy,petrol,4.38,2017,121239,2018,4.608908,1.739222e+07,0.10,141.502350,-6.624534,2.914989,171,3.900000,2.785895,31.650,-15.8250
2333,Audi A4,premium,petrol,4.30,2016,107793,2020,4.683793,1.174052e+07,0.10,155.000000,-8.582467,-18.805856,169,1.935294,1.631115,55.181,-27.5905
2334,Kia Rio,economy,petrol,3.88,2015,80234,2019,4.655345,1.202022e+07,0.10,104.180940,-0.778524,-7.651776,172,2.782353,2.311168,106.793,-53.3965
2335,Renault Sandero,standart,petrol,4.50,2014,60048,2020,4.638333,1.788307e+07,0.10,200.000000,2.464975,0.468718,171,3.441176,2.899751,51.144,-25.5720


0       0
1       2
2       7
3       4
4       4
       ..
2332    8
2333    3
2334    7
2335    0
2336    3
Length: 2337, dtype: int32

### Для удобства работы был создан абстрактный класс, на основе которого реализованы дочерние классы для обучения и подбора гиперпараметров разных моделей. В классе есть стандартные методы fit, predict. get_score, позволяющий получить скор при к-фолдном обучении. optimize_hyperparameters, с помощью которого можно оптимизировать гиперпараметры

In [3]:
class AbstractModel(ABC):  
    """  
    Абстрактный класс для создания моделей машинного обучения с использованием   
    метода оптимизации гиперпараметров Optuna. Класс предоставляет базовые   
    функции для обучения и предсказаний модели, а также для оценки и   
    оптимизации гиперпараметров.  

    Атрибуты:  
        best_params : dict
            Лучшие параметры найденные в результате оптимизации.  
        best_value : float 
            Лучшее значение метрики, полученное в ходе оптимизации.  
        study : optuna.Study 
            Объект изучения для оптимизации гиперпараметров.  
    Свойства:
        optuna_parametrs : dict
            Параметры для оптимизации с помощью Optuna
    """  
    
    def __init__(self) -> None:  
        """  
        Инициализация атрибутов модели.  
        """  
        self._optuna_params = {}
        self._fit_params = {}
        self.best_params = {}
        self.best_value = -1
        self.study = None
    
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series) -> None: 
        """  
        Обучение модели на тренировочных данных.  

        Параметры:  
            X_train : pd.DataFrame 
                Обучающие данные.  
            y_train : pd.Series 
                Целевые значения для обучения.  
        """  
        X_train = copy.copy(X_train)
        
        if self.preprocessing:
            X_train = self.preprocessing(X_train)
        self.model.fit(X_train, y_train)  

    def predict(self, X_test: pd.DataFrame) -> np.ndarray:   
        """  
        Предсказание на тестовых данных.  

        Параметры:  
            X_test : pd.DataFrame | np.array
                Данные для предсказания.  

        Возвращает предсказанные значения в виде np.array.
        """  
        X_test = copy.copy(X_test)
        
        if self.preprocessing:
            X_test = self.preprocessing(X_test)
        self.result = self.model.predict(X_test)  
        return self.result  

    def get_score(self, 
                  X: pd.DataFrame, 
                  y: pd.Series, 
                  X_test: pd.DataFrame, 
                  y_test: pd.Series, 
                  random_state: int = 42, 
                  n_splits: int = 5) -> list[float]:    
         """  
        Оценка модели с использованием кросс-валидации.  

        Параметры:  
            X : pd.DataFrame
                Исследуемый датасет. 
            y : pd.Series
                Целевые значения.  
            X_test : pd.DataFrame
                Данные для тестирования.  
            y_test : pd.Series
                Целевые значения для тестирования.  
            random_state : int
                Фиксирует сид.  
            n_splits : int
                Число фолдов для кросс-валидации.  

        Возвращает:  
            list: Список значений метрики.  
        """  
         X = copy.copy(X)
         y = copy.copy(y)
         X_test = copy.copy(X_test)
         y_test = copy.copy(y_test)
        
         X.reset_index(drop=True, inplace=True)
         y.reset_index(drop=True, inplace=True)
        
         scores = []         
        
         X = self.preprocessing(X)
         X_test = self.preprocessing(X_test)
        
         kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)  
         for train_index, test_index in kf.split(X):  
            
            X_train, X_val = X.loc[train_index], X.loc[test_index]  
            y_train, y_val = y[train_index], y[test_index]  
            
            model = self.model(**self._base_params)  
            
            model.fit(**self.get_fit_parametrs(X_train, y_train, X_val, y_val))  
            score = recall_score(y_test, model.predict(X_test), average='macro')  
            scores.append(score)
         return scores

    def objective(self, 
                  trial: optuna.Trial,
                  X: pd.DataFrame, 
                  y: pd.Series, 
                  n_splits: int, 
                  random_state: int):  
        """  
        Целевая функция для оптимизации гиперпараметров.  

        Параметры:  
            trial : optuna.Trial
                Единичный объект испытания Optuna.  
            X : pd.DataFrame
                Исследуемый датасет.  
            y : pd.Series
                Целевые значения.  
            n_splits : int
                Число фолдов.  
            random_state : int
                Случайное состояние для воспроизводимости.  

        Возвращает:  
            float: Средняя оценка модели.  
        """  
        scores = []  
          
        X = self.preprocessing(X)

        optuna_params = self._get_optuna_params(trial)  
        
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state) 
        for train_index, val_index in kf.split(X):  
            X_train, X_val = X.loc[train_index], X.loc[val_index]  
            y_train, y_val = y[train_index], y[val_index]  
            
            model_optuna = self.model(**optuna_params)  
            
            if self.get_pruner(trial):
                model_optuna.fit(**self.get_fit_parametrs(X_train, y_train, X_val, y_val), callbacks=[self.pruner])  
            else:
                model_optuna.fit(**self.get_fit_parametrs(X_train, y_train, X_val, y_val))
            score = recall_score(y_val, model_optuna.predict(X_val), average='macro')  
            scores.append(score)                  

        return np.mean(scores) - np.std(scores)  

    def _get_optuna_params(self, trial: optuna.Trial):  
        """  
        Получение параметров оптимизации для Optuna.  

        Параметры:  
            trial (optuna.Trial): Объект попытки от Optuna.  

        Возвращает параметры для Optuna в виде dict.
        """  
        params = {**self._base_params}  

        for param_name, param_props in self.optuna_parametrs.items():  
            
            if param_props['type'] == 'categorical':
                suggest_func = getattr(trial, f'suggest_{param_props["type"]}')  
                params[param_name] = suggest_func(param_name, param_props['categories'])
            else:
                suggest_func = getattr(trial, f'suggest_{param_props["type"]}')  
                params[param_name] = suggest_func(param_name, param_props['low'], param_props['high'])  

        return params  
    
    

    def optimize_hyperparameters(self, 
                                 X: pd.DataFrame, 
                                 y: pd.Series, 
                                 n_splits: int = 5, 
                                 n_trials: int = 500, 
                                 random_state: int = 42, 
                                 pruner: optuna.pruners = optuna.pruners.MedianPruner()) -> dict[str, Any]:  
        """  
        Оптимизация гиперпараметров с помощью Optuna.  

        Параметры:  
            X : pd.DataFrame
                 Исследуемый датасет.  
            y : pd.Series
                Целевые значения.  
            n_splits : int
                Число фолдов.  
            n_trials : int
                Число попыток для оптимизации.  
            random_state : int
                Случайное состояние для воспроизводимости.  
            pruner : optuna.Pruner
                Принтер для оптимизации.  

        Возвращает лучшие параметры и значение, полученные в ходе оптимизации в виде tuple.
        """  
        X = copy.copy(X)
        y = copy.copy(y) 
        
        X.reset_index(drop=True, inplace=True)
        y.reset_index(drop=True, inplace=True)

        study = optuna.create_study(direction='maximize', pruner=pruner)  
        study.optimize(lambda trial: self.objective(trial, X, y, n_splits, random_state), 
                       n_trials=n_trials, 
                       show_progress_bar=True) 
        
        if study.best_value > self.best_value:
            self.best_params = study.best_params
            self.study = study
            
        return study.best_params, study.best_value
    
    @abstractmethod  
    def get_fit_parametrs(self, 
                          X_train: pd.DataFrame,
                          y_train: pd.Series, 
                          X_val: pd.DataFrame, 
                          y_val: pd.Series) -> dict[str, Any]:
        """  
        Получение параметров для обучения модели.  

        Параметры:  
            X_train : pd.DataFrame 
                Обучающие данные.  
            y_train : pd.Series   
                Целевые значения.  
            X_val : pd.DataFrame 
                Валидационные данные.  
            y_val : pd.Series  
                Валидационные целевые значения.  
        
        Возвращает:  
            dict: Параметры для обучения модели.  
        """  
        pass
    
    @abstractmethod
    def __name__(self) -> str:
        """  
        Возвращает имя модели.  
        """  
        pass
    
    @abstractmethod
    def preprocessing(self) -> pd.DataFrame:
        """  
        Предобработка данных.  

        Возвращает обработанный датасет X.  
        """  
        pass
    
    def get_model(self) -> Any:  
        """  
        Получение модели.  

        Возвращает:  
            object: Экземпляр модели.  
        """  
        return self.model  

    @property  
    def optuna_parametrs(self) -> dict[str, Any]: 
        """  
        Возвращает текущие параметры для оптимизации в виде dict.
        """  
        return self._optuna_params  

    @optuna_parametrs.setter  
    def optuna_parametrs(self, value: dict[str, Any]) -> None:  
        """  
        Установка новых параметров для оптимизации.  

        Параметры:  
            value : dict
                Новые параметры для оптимизации.  
        """  
        self._optuna_params = value
    
    @property
    def fit_parametrs(self) -> dict[str, Any]:  
        """  
        Возвращает текущие параметры для обучения в виде dict.   
        """  
        return self._fit_params
    
    @fit_parametrs.setter
    def fit_parametrs(self, value: dict[str, Any]) -> None:
        """  
        Установка новых параметров для обучения.  

        Параметры:  
            value : dict
                Новые параметры для обучения.  
        """  
        self._fit_params = value    

### Для хранения настраиваемых оптуной параметров созданы два датакласса. Parameter, хранящий тип данных, подающихся в оптуну и максимальное/минимальное значение для числовой параметров или список для категориальных параметров. Второй класс - ModelParameter, в нем хранятся экземпляры класса Parameter

In [4]:
@dataclass  
class Parameter:  
    """  
    Класс Parameter представляет параметр с определенным типом, границами/категориями.  
    
    Атрибуты:  
        type : str 
            Тип параметра.  
        low : float 
            Нижняя граница параметра, by default None.  
        high : float
            Верхняя граница параметра by default None.  
        categories : List[str]
            Список категорий для данного параметра by default list().  
    """  
    type: str  
    low: float = None  
    high: float = None  
    categories: List[str] = field(default_factory=list)  
    
    def update(self, **kwargs):  
        """  
        Обновляет атрибуты класса с использованием переданных именованных аргументов.  
        
        Аргументы : **kwargs
            Именованные аргументы, соответствующие атрибутам класса.  

        """  
        setattr(self, key, value) 
    

In [5]:
@dataclass  
class ModelParams: 
    """  
    Класс ModelParams представляет набор параметров модели.  
    
    Атрибуты:  
        params : Dict[str, Parameter] 
        Словарь, содержащий параметры модели, ключ – название параметра, значение – объект Parameter.  
    """  
    params: Dict[str, Parameter] = field(default_factory=dict)  
        
    def to_dict(self) -> Dict[str, Any]:  
        """  
        Преобразует параметры модели в словарь.  
        
        Возвращает:  Dict[str, Any]
            Словарь, где ключи – названия параметров, а значения – их атрибуты в виде словаря.  
        """  
        
        return {name: param.__dict__ for name, param in self.params.items()} 
    
    def update(self, **kwargs):  
        setattr(self, key, value) 

### Функция, выдающая гиперпараметры для оптимизации 

In [6]:
def get_optuna_parametrs(model: Any) -> ModelParams:  
    """  
    Получает параметры для настройки модели машинного обучения для использования с библиотекой Optuna.  
    
    Аргументы:  
        model (Any): Объект модели машинного обучения, от которого будет извлечено название модели.  
    
    Возвращает:  
        ModelParams: Объект ModelParams, содержащий параметры для настройки модели.   
                     Параметры представлены в виде словаря, где каждое имя параметра   
                     связано с объектом Parameter, описывающим тип и диапазоны значений.  
    
    Исключения:  
        ValueError: Выбрасывается, если модель с указанным именем не найдена в словаре параметров.  
    
    Пример использования:  
        params = get_optuna_parametrs(CatBoostModel)  
        print(params)  # Выводит параметры для CatBoostModel в виде словаря.  
    """  
    model_name = model.__name__()
    parameters = {  
        'CatBoostModel': ModelParams(params={  
            'learning_rate': Parameter(type='float', low=0.01, high=0.5),  
            'l2_leaf_reg': Parameter(type='int', low=2, high=50),  
            'colsample_bylevel': Parameter(type='float', low=0.01, high=0.8),  
            'depth': Parameter(type='int', low=4, high=6),  
            'auto_class_weights': Parameter(type='categorical', categories=['SqrtBalanced', 'Balanced', 'None']),  
            'boosting_type': Parameter(type='categorical', categories=['Ordered', 'Plain']),  
            'bootstrap_type': Parameter(type='categorical', categories=['Bayesian', 'Bernoulli', 'MVS']),  
        }),  
        
        'RandomForestModel': ModelParams(params={  
            'n_estimators': Parameter(type='int', low=100, high=500),  
            'criterion': Parameter(type='categorical', categories=['gini', 'entropy']),  
            'max_features': Parameter(type='categorical', categories=['sqrt', 'log2', 'auto']),  
            'max_depth': Parameter(type='int', low=3, high=40),  
        }),
        
        'LgbmModel': ModelParams(params={
            'learning_rate': Parameter(type='float', low=0.001, high=0.3),
            'num_leaves': Parameter(type='int', low=20, high=5000), 
            'extra_trees': Parameter(type='categorical', categories=['True', 'False']),
            'reg_alpha': Parameter(type='float', low=0, high=100),
            'reg_lambda': Parameter(type='float', low=0, high=100),
            "subsample": Parameter(type='float', low=0.05, high=1.0),
            "colsample_bytree": Parameter(type='float', low=0.05, high=1.0),
            "min_data_in_leaf": Parameter(type='int', low=1, high=100),
            'min_gain_to_split': Parameter(type='int', low=0, high=15)
        }),
        
        'XgboostModel': ModelParams(params={
            'learning_rate': Parameter(type='float', low=0.01, high=1),
            'max_depth': Parameter(type='int', low=3, high=10),
            'subsample': Parameter(type='float', low=0.5, high=1.0),
            'colsample_bytree': Parameter(type='float', low=0.5, high=1.0),
            'min_child_weight': Parameter(type='int', low=1, high=40),
            'gamma': Parameter(type='float', low=0, high=0.5),
            'lambda': Parameter(type='float', low=0, high=0.5),
            'alpha': Parameter(type='float', low=0, high=0.5),
            'n_estimators': Parameter(type='int', low=100, high=200),

        }),
        
        'SVMModel': ModelParams(params={
            'C': Parameter(type='float', low=0.01, high=100),
            'degree': Parameter(type='int', low=2, high=5),
            'gamma': Parameter(type='categorical', categories=['scale', 'auto']),
            'coef0': Parameter(type='float', low=-0.7, high=0.7),
            
        }),

         'MLPModel': ModelParams(params={  
             'activation': Parameter(type='categorical', categories=['logistic', 'tanh', 'relu']),  
             'alpha': Parameter(type='loguniform', low=1e-5, high=1e-1),  
             'learning_rate_init': Parameter(type='loguniform', low=1e-5, high=1e-1)  
         }),
        'KNNModel': ModelParams(params={  
            'n_neighbors': Parameter(type='int', low=3, high=20),
            'weights': Parameter(type='categorical', categories=['distance', 'uniform']),
            'metric': Parameter(type='categorical', categories=['euclidean', 'manhattan', 'chebyshev']),
            'algorithm': Parameter(type='categorical', categories=['ball_tree', 'auto', 'kd_tree', 'brute'])
        })
    }  
    


    model_params = parameters.get(model_name)
    if model_params:  

        # Установка зависимых параметров на основе значений других параметров
        set_dependent_parameters(model_params.params, model) 
        
        # Вовзрат параметров в виде словаря
        return model_params.to_dict()

    raise ValueError(f"Модель '{model_name}' не найдена.")  
    

### Добавление зависимых параметров для настройки моделей

In [7]:
def set_dependent_parameters(params_dict: Dict[str, Parameter], model: Any) -> None:  
    """  
    Устанавливает зависимые параметры для заданной модели машинного обучения на основе  
    её атрибутов. Функция модифицирует переданный словарь параметров, добавляя  
    дополнительные параметры в зависимости от текущей конфигурации модели. 
    """
    model_name = model.__name__()
    if model_name == 'LgbmModel':
        if model._base_params['boosting_type'] != 'goss':         
            params_dict['bagging_freq'] = Parameter(type='int', low=1, high=10)
            params_dict['bagging_fraction'] = Parameter(type='float', low=0.5, high=1.0)

    if model_name == 'XgboostModel':  
        if model._base_params['booster'] == 'dart':  
            params_dict['rate_drop'] = Parameter(type='float', low=0.0, high=0.1)  
            params_dict['skip_drop'] = Parameter(type='float', low=0.0, high=0.5)


## Далее будут созданы модели и подобраны гиперпараметры по следующему шаблону:
- ### Создание класса на основе AbstractModel. Обязательно должны быть следующие функции: get_fit_parametrs, \_\_name\_\_  , preprocessing
- ### Обучение модели с дефолтными параметрами и к-фолдной валидацией
- ### Подбор гиперпараметров с помощью Optuna
### Будут созданы классы для следующих моделей: Catboost, Xgboost, LGBM, MLP, SVM, KNN. Достаточно просто реализовать и другие модели
### Optuna оптимизирует разницу среднего значения и среднеквадратичного отклонения скоров, полученных при к-фолдной валидации (по дефолту на пяти фолдах)
### P.S. В конечной версии ноутбука оптимизация будет запущена на 1 trial для иллюстрации, однако при оптимизации количество испытаний было от 500 до 2500 в зависимости от модели

# Catboost

In [8]:
class CatBoostModel(AbstractModel):
    
    def __init__(self, **kwargs):       
        super().__init__()
        self._base_params = kwargs
        self.model = CatBoostClassifier
        
    def get_fit_parametrs(self, X_train, y_train, X_val, y_val):
        
        parametrs = {
            'X': Pool(data=X_train, label=y_train, cat_features=categorical_features), 
            'eval_set': Pool(data=X_val, label=y_val, cat_features=categorical_features),
            'early_stopping_rounds': 20,    
        }
        
        return parametrs
    
    def __name__(self):
        return 'CatBoostModel'
    
    def get_pruner(self, trial):
        self.pruner = optuna.integration.CatBoostPruningCallback(trial, "MultiClass")
        
    def preprocessing(self, X):
        return X

In [9]:
invariant_catboost_params = {
    'loss_function': 'MultiClass',
    'custom_metric': 'Recall',
    'cat_features': categorical_features,
    'task_type': 'CPU',
    'thread_count': -1,
    'silent': True,
    'random_state': 1
}

catboost_default = CatBoostModel(**invariant_catboost_params)

In [10]:
catboost_scores_default = catboost_default.get_score(X_train, y_train, X_test, y_test)
print('Catboost; значения до подбора гиперпараметров, Recall:', 
      round(np.mean(catboost_scores_default), 4), 
      'std:', round(np.std(catboost_scores_default), 4))

Catboost; значения до подбора гиперпараметров, Recall: 0.9221 std: 0.0053


In [11]:
catboost_default.optuna_parametrs = get_optuna_parametrs(catboost_default) 

best_catboost_params, best_catboost_score = catboost_default.optimize_hyperparameters(X_train, 
                                                                                      y_train, 
                                                                                      n_trials=1, 
                                                                                      random_state=1)

  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
best_catboost_params = {'learning_rate': 0.25015967138404244,
 'l2_leaf_reg': 18,
 'colsample_bylevel': 0.5121440134918076,
 'depth': 5,
 'auto_class_weights': 'SqrtBalanced',
 'boosting_type': 'Plain',
 'bootstrap_type': 'MVS'}

# Random forest

In [13]:
class RandomForestModel(AbstractModel):  
    
    def __init__(self, **kwargs):          
        super().__init__()
        self._base_params = kwargs
        self.model = RandomForestClassifier
        
    def get_fit_parametrs(self, X_train, y_train, X_val, y_val):
        return {'X': X_train, 'y': y_train}
    
    def __name__(self):
        return 'RandomForestModel'
    
    def get_pruner(self, trial):
        pass
    
    def preprocessing(self, X):
        for cat in categorical_features:
            lbl = LabelEncoder()
            X[cat] = lbl.fit_transform(X[cat].astype(str))
            X[cat] = X[cat].astype('category')
        scaler = StandardScaler()
        return pd.DataFrame(scaler.fit_transform(X))


In [14]:
invariant_forest_params = {
    'n_jobs': -1,
    'random_state': 2,
    'verbose': 0
}

randomforest_default = RandomForestModel(**invariant_forest_params)

In [15]:
randomforest_scores_default = randomforest_default.get_score(X_train, y_train, X_test, y_test)
print('Random forest; значения до подбора гиперпараметров, Recall:', 
      round(np.mean(randomforest_scores_default), 4), 
      'std:', round(np.std(randomforest_scores_default), 4))

Random forest; значения до подбора гиперпараметров, Recall: 0.9131 std: 0.0069


In [16]:
randomforest_default.optuna_parametrs = get_optuna_parametrs(randomforest_default) 

best_randomforest_params, best_randomforest_score = randomforest_default.optimize_hyperparameters(X_train, 
                                                                                                  y_train, 
                                                                                                  n_trials=1, 
                                                                                                  random_state=1
                                                                                                 )

  0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
best_randomforest_params = {'n_estimators': 146,
 'criterion': 'gini',
 'max_features': 'sqrt',
 'max_depth': 36}

# Xgboost

In [18]:
class XgboostModel(AbstractModel):
    
    def __init__(self, **kwargs):          
        super().__init__()
        self._base_params = kwargs
        self.model = XGBClassifier
        
    def get_fit_parametrs(self, X_train, y_train, X_val, y_val):
        return {'X': X_train, 'y': y_train, 'eval_set': [(X_val, y_val)], 'verbose': False}
    
    def __name__(self):
        return 'XgboostModel'
    
    def get_pruner(self, trial):
        self.pruner = optuna.integration.XGBoostPruningCallback(trial, 'validation_0-recall')
        
    def preprocessing(self, X):
        for cat in categorical_features:
            lbl = LabelEncoder()
            X[cat] = lbl.fit_transform(X[cat].astype(str))
            X[cat] = X[cat].astype('category')
        return X

In [19]:
def recall(y_true, y_pred):
        return recall_score(y_true, y_pred, average="macro")
    
invariant_xgboost_params = {
    'objective': 'multi:softmax',
    'tree_method': 'hist',
    'seed': 42,
    'num_class': 9,
    'enable_categorical': True,    
    'eval_metric': recall,
}

xgboost_default = XgboostModel(
    booster='gbtree',
    random_state=3,
    **invariant_xgboost_params
)

In [20]:
xgboost_scores_default = xgboost_default.get_score(X_train, y_train, X_test, y_test)
print('Xgboost в gbtree режиме; значения до подбора гиперпараметров, Recall::', 
      round(np.mean(xgboost_scores_default), 4), 
      'std:', round(np.std(xgboost_scores_default), 4))

Xgboost в gbtree режиме; значения до подбора гиперпараметров, Recall:: 0.9206 std: 0.0039


In [21]:
xgboost_default.optuna_parametrs = get_optuna_parametrs(xgboost_default) 

best_xgboost_params, best_xgboost_score = xgboost_default.optimize_hyperparameters(X_train, 
                                                                                   y_train, 
                                                                                   n_trials=1,
                                                                                  random_state=3)

  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
best_xgboost_params = {'learning_rate': 0.36210437713740196,
 'max_depth': 9,
 'subsample': 0.9899372761932896,
 'colsample_bytree': 0.5000988712156675,
 'min_child_weight': 5,
 'gamma': 0.43736138023436366,
 'lambda': 0.2078567891538763,
 'alpha': 0.2842233805793963,
 'n_estimators': 149}

In [23]:
xgboost_default_dart = XgboostModel(
    booster='dart',
    **invariant_xgboost_params,
    random_state=4
)

In [24]:
xgboost_scores_default_dart = xgboost_default_dart.get_score(X_train, y_train, X_test, y_test)
print('Xgboost в dart режиме; значения до подбора гиперпараметров, Recall:', 
      round(np.mean(xgboost_scores_default_dart), 4),
      'std:', round(np.std(xgboost_scores_default_dart), 4))

Xgboost в dart режиме; значения до подбора гиперпараметров, Recall: 0.9206 std: 0.0039


In [25]:
xgboost_default_dart.optuna_parametrs = get_optuna_parametrs(xgboost_default_dart)

best_xgboost_params_dart, best_xgboost_score_dart = xgboost_default_dart.optimize_hyperparameters(
                                                                                                X_train, 
                                                                                                y_train, 
                                                                                                n_trials=1,
                                                                                                random_state=4
                                                                                                )

  0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
best_xgboost_params_dart = {'learning_rate': 0.3008374929907845,
 'max_depth': 3,
 'subsample': 0.8879729845591383,
 'colsample_bytree': 0.8584476440096315,
 'min_child_weight': 6,
 'gamma': 0.4216455695798721,
 'lambda': 0.49427853359506246,
 'alpha': 0.3962595442609055,
 'n_estimators': 149,
 'rate_drop': 0.03812530172660336,
 'skip_drop': 0.14630893743750664}

# LightGBM

In [27]:
class LgbmModel(AbstractModel):
    
    def __init__(self, **kwargs):          
        super().__init__()
        self._base_params = kwargs
        self.model = LGBMClassifier
        
    def get_fit_parametrs(self, X_train, y_train, X_val, y_val):
        return {'X': X_train, 'y':y_train, 'eval_set': (X_val, y_val), 'eval_metric': self.recall_metric}
    
    def recall_metric(self, y_true, y_pred):   
        y_pred = np.argmax(y_pred, axis=1)
        return 'recall', recall_score(y_true, y_pred, average='macro'), True 
    
    def __name__(self):
        return 'LgbmModel'
    
    def get_pruner(self, trial):
        self.pruner = optuna.integration.LightGBMPruningCallback(trial, 'recall')
        
    def preprocessing(self, X):
        return X

In [28]:
invariant_lgbm_params = {
    'objective': 'multiclass',
    'num_class': 9,
    'seed': 42,
    'cat_feature': [0, 1, 2],
    'verbose': -1,
}

In [29]:
lgbm_default = LgbmModel(
    boosting_type='gbdt',
    random_state=5,
    **invariant_lgbm_params
)

In [30]:
lgbm_scores_default = lgbm_default.get_score(X_train, y_train, X_test, y_test)
print('LGBM в gbdt режиме; значения до подбора гиперпараметров, Recall:', 
      round(np.mean(lgbm_scores_default), 4), 
      'std:', round(np.std(lgbm_scores_default), 4))

  File "C:\Users\savus\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 217, in _count_physical_cores
    raise ValueError(


LGBM в gbdt режиме; значения до подбора гиперпараметров, Recall: 0.9161 std: 0.0044


In [31]:
lgbm_default.optuna_parametrs = get_optuna_parametrs(lgbm_default) 

best_lgbm_params, best_lgbm_score = lgbm_default.optimize_hyperparameters(X_train, 
                                                                          y_train, 
                                                                          n_trials=1, 
                                                                          random_state=5)

  0%|          | 0/1 [00:00<?, ?it/s]

In [32]:
best_lgbm_params = {'learning_rate': 0.27938850966497325,
 'num_leaves': 1075,
 'extra_trees': 'False',
 'reg_alpha': 1.7690055617828495,
 'reg_lambda': 82.85143348712724,
 'subsample': 0.12117641840325521,
 'colsample_bytree': 0.974895774239529,
 'min_data_in_leaf': 1,
 'min_gain_to_split': 0,
 'bagging_freq': 6,
 'bagging_fraction': 0.9891316013437009}

In [33]:
lgbm_default_dart = LgbmModel(
    boosting_type='dart',
    random_state=6,
    **invariant_lgbm_params
)

In [34]:
lgbm_scores_dart = lgbm_default_dart.get_score(X_train, y_train, X_test, y_test)
print('LGBM в dart режиме; значения до подбора гиперпараметров, Recall:', 
      round(np.mean(lgbm_scores_dart), 4),
      'std:', round(np.std(lgbm_scores_dart), 4))

LGBM в dart режиме; значения до подбора гиперпараметров, Recall: 0.9154 std: 0.0027


In [35]:
lgbm_default_dart.optuna_parametrs = get_optuna_parametrs(lgbm_default_dart) 

best_lgbm_dart_params, best_lgbm_dart_score = lgbm_default_dart.optimize_hyperparameters(X_train, 
                                                                                         y_train, 
                                                                                         n_trials=1, 
                                                                                         random_state=6)

  0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
best_lgbm_dart_params = {'learning_rate': 0.2945353365859827,
 'num_leaves': 2460,
 'extra_trees': 'False',
 'reg_alpha': 0.04691960161462472,
 'reg_lambda': 76.56966835042759,
 'subsample': 0.2175658527267273,
 'colsample_bytree': 0.8505709820546443,
 'min_data_in_leaf': 3,
 'min_gain_to_split': 1,
 'bagging_freq': 10,
 'bagging_fraction': 0.7532726845342235}

In [37]:
lgbm_default_goss = LgbmModel(
    boosting_type='goss',
    random_state=7,
    **invariant_lgbm_params
)

In [38]:
lgbm_scores_goss = lgbm_default_goss.get_score(X_train, y_train, X_test, y_test)
print('LGBM в goss режиме; значения до подбора гиперпараметров, Recall:', 
      round(np.mean(lgbm_scores_goss), 4),
      'std:', round(np.std(lgbm_scores_goss), 4))

LGBM в goss режиме; значения до подбора гиперпараметров, Recall: 0.9131 std: 0.0033


In [39]:
lgbm_default_goss.optuna_parametrs = get_optuna_parametrs(lgbm_default_goss)

best_lgbm_goss_params, best_lgbm_goss_score = lgbm_default_goss.optimize_hyperparameters(X_train, 
                                                                                         y_train, 
                                                                                         n_trials=1, 
                                                                                         random_state=7)

  0%|          | 0/1 [00:00<?, ?it/s]

In [40]:
best_lgbm_goss_params = {'learning_rate': 0.1444019114621476,
 'num_leaves': 2221,
 'extra_trees': 'False',
 'reg_alpha': 15.418547774446296,
 'reg_lambda': 61.076829128367024,
 'subsample': 0.7436898759017979,
 'colsample_bytree': 0.6315555709656593,
 'min_data_in_leaf': 1,
 'min_gain_to_split': 0}

# MLP

In [41]:
class MLPModel(AbstractModel):
    
    def __init__(self, **kwargs):          
        super().__init__()
        self._base_params = kwargs
        self.model = MLPClassifier
        
    def get_fit_parametrs(self, X_train, y_train, X_val, y_val):
        return {'X': X_train, 'y':y_train}
    
    def __name__(self):
        return 'MLPModel'
    
    def get_pruner(self, trial):
        pass
    
    def preprocessing(self, X):
        for cat in categorical_features:
            lbl = LabelEncoder()
            X[cat] = lbl.fit_transform(X[cat].astype(str))
            X[cat] = X[cat].astype('category')
            
        scaler = StandardScaler()
        return pd.DataFrame(scaler.fit_transform(X)) 

In [42]:
invariant_mlp_params = {
    'max_iter': 1000, 
    'random_state': 8, 
    'hidden_layer_sizes': (100, 30)
}

In [43]:
mlp_default = MLPModel(**invariant_mlp_params)

mlp_scores_default = mlp_default.get_score(X_train, y_train, X_test, y_test, random_state=8)

print('MLP; значения до подбора гиперпараметров, Recall:', 
      round(np.mean(mlp_scores_default), 4), 
      'std:', round(np.std(mlp_scores_default), 4))

MLP; значения до подбора гиперпараметров, Recall: 0.8881 std: 0.0062


In [44]:
mlp_default.optuna_parametrs = get_optuna_parametrs(mlp_default)

best_mlp_params, best_mlp_score = mlp_default.optimize_hyperparameters(X_train, 
                                                                       y_train, 
                                                                       n_trials=1, 
                                                                       random_state=8)

  0%|          | 0/1 [00:00<?, ?it/s]

In [45]:
best_mlp_params = {'activation': 'tanh',
 'alpha': 0.0008350069445472012,
 'learning_rate_init': 0.0001263425120723001}

# SVM

In [46]:
class SVMModel(AbstractModel):
    
    def __init__(self, **kwargs):          
        super().__init__()
        self._base_params = kwargs
        self.model = SVC
        
    def get_fit_parametrs(self, X_train, y_train, X_val, y_val):
        return {'X': X_train, 'y':y_train}
    
    def __name__(self):
        return 'SVMModel'
    
    def get_pruner(self, trial):
        pass
    
    def preprocessing(self, X):
        le = LabelEncoder()
        for col in categorical_features:
            X[col] = le.fit_transform(X[col])
        scaler = StandardScaler() 
        return pd.DataFrame(scaler.fit_transform(X)) 

In [47]:
invariant_svm_linear_params = {
    'kernel': 'linear',
    'random_state': 9,
}

In [48]:
svm_linear_default = SVMModel(**invariant_svm_linear_params)

svm_linear_scores_default = svm_linear_default.get_score(X_train, y_train, X_test, y_test, random_state=9)

print('SVM Linear; значения до подбора гиперпараметров, Recall:', 
      round(np.mean(svm_linear_scores_default), 4),
      'std:', round(np.std(svm_linear_scores_default), 4))

SVM Linear; значения до подбора гиперпараметров, Recall: 0.9094 std: 0.0065


In [49]:
svm_linear_default.optuna_parametrs = get_optuna_parametrs(svm_linear_default)

best_svm_params, best_svm_score = svm_linear_default.optimize_hyperparameters(X_train, 
                                                                              y_train, 
                                                                              n_trials=1, 
                                                                              random_state=9)

  0%|          | 0/1 [00:00<?, ?it/s]

In [50]:
best_svm_linear_params = {'C': 0.18811384485519722,
 'gamma': 'auto',
 'coef0': -0.32221472375493637}

In [51]:
invariant_svm_poly_params = {
    'kernel': 'poly',
    'random_state': 10
}

In [52]:
svm_poly_default = SVMModel(**invariant_svm_poly_params)

svm_poly_scores_default = svm_poly_default.get_score(X_train, y_train, X_test, y_test, random_state=10)

print('SVM Poly; значения до подбора гиперпараметров, Recall:', 
      round(np.mean(svm_poly_scores_default), 4),
      'std:', round(np.std(svm_poly_scores_default), 4))

SVM Poly; значения до подбора гиперпараметров, Recall: 0.798 std: 0.0145


In [53]:
svm_poly_default.optuna_parametrs = get_optuna_parametrs(svm_poly_default)

best_svm_params, best_svm_score = svm_poly_default.optimize_hyperparameters(X_train, 
                                                                            y_train, 
                                                                            n_trials=1, 
                                                                            random_state=10)

  0%|          | 0/1 [00:00<?, ?it/s]

In [54]:
best_svm_poly_params = {'C': 1.3054435718959585,
 'degree': 2,
 'gamma': 'auto',
 'coef0': 0.4492718754629773}

# KNN

In [55]:
class KNNModel(AbstractModel):   
    def __init__(self, **kwargs):          
        super().__init__()
        self._base_params = kwargs
        self.model = KNeighborsClassifier
        
    def get_fit_parametrs(self, X_train, y_train, X_val, y_val):
        return {'X': X_train, 'y':y_train}
    
    def __name__(self):
        return 'KNNModel'
    
    def get_pruner(self, trial):
        pass
    
    def preprocessing(self, X):
        le = LabelEncoder()
        for col in categorical_features:
            X[col] = le.fit_transform(X[col])
        scaler = StandardScaler() 
        return pd.DataFrame(scaler.fit_transform(X)) 

In [56]:
knn_default = KNNModel(n_jobs=-1)

knn_scores_default = knn_default.get_score(X_train, y_train, X_test, y_test, random_state=10)

print('KNN; значения до подбора гиперпараметров, Recall:', 
      round(np.mean(knn_scores_default), 4),
      'std:', round(np.std(knn_scores_default), 4))

KNN; значения до подбора гиперпараметров, Recall: 0.6699 std: 0.0088


In [57]:
knn_default.optuna_parametrs = get_optuna_parametrs(knn_default)

best_knn_params, best_knn_score = knn_default.optimize_hyperparameters(X_train, 
                                                                       y_train, 
                                                                       n_trials=1,                                            
                                                                       random_state=10)

  0%|          | 0/1 [00:00<?, ?it/s]

In [58]:
best_knn_params = {'n_neighbors': 16,
 'weights': 'distance',
 'metric': 'manhattan',
 'algorithm': 'kd_tree'}

# Стекинг

### Дефолтный стекинг состоит из 12 моделей:
- #### Catboost
- #### Xgboost
- #### Xgboost в dart режиме
- #### LGBM 
- #### LGBM в dart режиме
- #### LGBM в goss режиме
- #### Random forest
- #### ExtraTrees
- #### SVM со степенью 2
- #### Линейный SVM 
- #### MLP
- #### KNN
### При необходимости модели пропускаются через трансформеры: 
- #### Числовые значения нормализуются, пропуски заполняются средними значениями
- #### Категориальные значения кодируются OHE, пропуски заполняются наиболее частыми значениями
### Очевидно, что моделей в стекинге много, одних градиентных бустингов шесть. Хоть гиперпараметры у бустингов разные (какие-то модели сильно регулиризованные, какие-то обучались в dart режимах и тд), но они наверняка будут между собой сильно коррелировать. После построения стекинга со всеми моделями придется найти оптимальную комбинацию. Так же после обучения стекинга можно посмотреть на улучшение скоров после подбора гиперпараметров для всех моделей.

In [59]:
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy='most_frequent')),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))])

In [60]:
numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer()),
    ("scaler", StandardScaler())
])

In [61]:
preprocessor = ColumnTransformer(transformers=[
    ("numerical", numerical_transformer, numerical_features),
    ("categorical", categorical_transformer, categorical_features)])

preprocessor

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['car_rating', 'year_to_start', 'riders',
                                  'year_to_work', 'mean_rating', 'distance_sum',
                                  'rating_min', 'speed_max',
                                  'user_ride_quality_median',
                                  'deviation_normal_mean', 'user_uniq',
                                  'mean_destroy_degree', 'std_destroy_degree',
                                  'absolute_difference',
                                  'relative_difference']),
                                ('categorical',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['model', 'car_type', 'fuel_type'])])

In [62]:
estimators = [
    ('Catboost', CatBoostClassifier(
                                    **best_catboost_params, 
                                    **invariant_catboost_params,
                                    )
    ),
    
    ('Lgbm_gbdt', LGBMClassifier(random_state=15,
                                **best_lgbm_params, 
                                **invariant_lgbm_params
                                )
    ),
    
    ('Lgbm_dart', LGBMClassifier(random_state=16,
                                **best_lgbm_dart_params, 
                                **invariant_lgbm_params
                                )
   ),
    
    ('Lgbm_goss', LGBMClassifier(random_state=17,
                                **best_lgbm_goss_params, 
                                **invariant_lgbm_params
                                )
    ),
    
    ('Xgboost', XGBClassifier(random_state=18,
                            **best_xgboost_params, 
                            **invariant_xgboost_params
                            )
    ),
    
    ('Xgboost_dart', XGBClassifier(random_state=19,
                                **best_xgboost_params_dart, 
                                **invariant_xgboost_params
                                )
    ),
    
    ('Random_forest', make_pipeline(preprocessor, RandomForestClassifier(
                                            **best_randomforest_params, 
                                            **invariant_forest_params
                                            )
                                   )
    ),
    
    ("ExtraTrees",  make_pipeline(preprocessor, ExtraTreesClassifier(n_estimators = 10_000, 
                                                                     max_depth = 6, 
                                                                     min_samples_leaf = 2, 
                                                                     bootstrap = True, 
                                                                     class_weight = 'balanced',
                                                                     random_state = 20, 
                                                                     verbose=False,
                                                                     n_jobs=-1,))),
    
    ("SVM_poly", make_pipeline(preprocessor, SVC(random_state=21, kernel='poly', **best_svm_poly_params))),
    
    ("SVM_linear", make_pipeline(preprocessor, SVC(random_state=22, kernel='linear', **best_svm_linear_params))),
    
    ("MLP", make_pipeline(preprocessor, MLPClassifier(**invariant_mlp_params, **best_mlp_params))),
    ('KNN',make_pipeline(preprocessor, KNeighborsClassifier (n_jobs=-1, **best_knn_params)))
]

stacking_classifier_full = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(verbose=False, random_state=24,),
    n_jobs=-1,
    verbose=False,
)

stacking_classifier_full

StackingClassifier(estimators=[('Catboost',
                                <catboost.core.CatBoostClassifier object at 0x0000014745970550>),
                               ('Lgbm_gbdt',
                                LGBMClassifier(bagging_fraction=0.9891316013437009,
                                               bagging_freq=6,
                                               cat_feature=[0, 1, 2],
                                               colsample_bytree=0.974895774239529,
                                               extra_trees='False',
                                               learning_rate=0.27938850966497325,
                                               min_data_in_leaf=1,
                                               min_gain_to_split=0, num_class=9,
                                               num_le...
                                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                                  ('onehot',
                                                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                                                  ['model',
                                                                                   'car_type',
                                                                                   'fuel_type'])])),
                                                ('kneighborsclassifier',
                                                 KNeighborsClassifier(algorithm='kd_tree',
                                                                      metric='manhattan',
                                                                      n_jobs=-1,
                                                                      n_neighbors=16,
                                                                      weights='distance'))]))],
                   final_estimator=LogisticRegression(random_state=24,
                                                      verbose=False),
                   n_jobs=-1, verbose=False)

In [63]:
stacking_classifier_full.fit(X_train, y_train)

StackingClassifier(estimators=[('Catboost',
                                <catboost.core.CatBoostClassifier object at 0x0000014745970550>),
                               ('Lgbm_gbdt',
                                LGBMClassifier(bagging_fraction=0.9891316013437009,
                                               bagging_freq=6,
                                               cat_feature=[0, 1, 2],
                                               colsample_bytree=0.974895774239529,
                                               extra_trees='False',
                                               learning_rate=0.27938850966497325,
                                               min_data_in_leaf=1,
                                               min_gain_to_split=0, num_class=9,
                                               num_le...
                                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                                  ('onehot',
                                                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                                                  ['model',
                                                                                   'car_type',
                                                                                   'fuel_type'])])),
                                                ('kneighborsclassifier',
                                                 KNeighborsClassifier(algorithm='kd_tree',
                                                                      metric='manhattan',
                                                                      n_jobs=-1,
                                                                      n_neighbors=16,
                                                                      weights='distance'))]))],
                   final_estimator=LogisticRegression(random_state=24,
                                                      verbose=False),
                   n_jobs=-1, verbose=False)

In [64]:
print(f'Recall стекинга из {len(estimators)} моделей:', recall_score(stacking_classifier_full.predict(X_test), y_test, average='macro'))

Recall стекинга из 12 моделей: 0.9364624497052341


In [65]:
corr_df = pd.DataFrame(index=[i for i in range(X_test.shape[0])])

for name, model in stacking_classifier_full.estimators:
    model.fit(X_train, y_train)
    corr_df[name] = model.predict(X_test)
    print(name, 'recall: ', round(recall_score(model.predict(X_test), y_test, average='macro'), 4))


corr_df.corr().style.background_gradient(cmap="RdYlGn")

Catboost recall:  0.9318
Lgbm_gbdt recall:  0.9347
Lgbm_dart recall:  0.9184
Lgbm_goss recall:  0.9255
Xgboost recall:  0.9225
Xgboost_dart recall:  0.9303
Random_forest recall:  0.9316
ExtraTrees recall:  0.9062
SVM_poly recall:  0.9084
SVM_linear recall:  0.9113
MLP recall:  0.9065
KNN recall:  0.8172


,Catboost,Lgbm_gbdt,Lgbm_dart,Lgbm_goss,Xgboost,Xgboost_dart,Random_forest,ExtraTrees,SVM_poly,SVM_linear,MLP,KNN
Catboost,1.000000,0.986459,0.971239,0.982111,0.970927,0.993823,0.974670,0.922663,0.947668,0.942856,0.964035,0.817514
Lgbm_gbdt,0.986459,1.000000,0.955531,0.983604,0.971197,0.981937,0.976125,0.932301,0.933319,0.931276,0.949698,0.823541
Lgbm_dart,0.971239,0.955531,1.000000,0.979304,0.969505,0.977324,0.961044,0.906644,0.951129,0.951668,0.938206,0.811194
Lgbm_goss,0.982111,0.983604,0.979304,1.000000,0.984116,0.982819,0.978811,0.934069,0.949950,0.949212,0.946120,0.832237
Xgboost,0.970927,0.971197,0.969505,0.984116,1.000000,0.973009,0.971327,0.948545,0.940157,0.966211,0.942016,0.846470
Xgboost_dart,0.993823,0.981937,0.977324,0.982819,0.973009,1.000000,0.975297,0.919295,0.949543,0.950106,0.955341,0.823359
Random_forest,0.974670,0.976125,0.961044,0.978811,0.971327,0.975297,1.000000,0.940426,0.937200,0.944648,0.954965,0.829503
ExtraTrees,0.922663,0.932301,0.906644,0.934069,0.948545,0.919295,0.940426,1.000000,0.902372,0.931491,0.914658,0.850069
SVM_poly,0.947668,0.933319,0.951129,0.949950,0.940157,0.949543,0.937200,0.902372,1.000000,0.959383,0.958949,0.821822
SVM_linear,0.942856,0.931276,0.951668,0.949212,0.966211,0.950106,0.944648,0.931491,0.959383,1.000000,0.938082,0.841879


### После подбора гиперпараметров скоры базовых моделей улучшились в среднем на 1,5-2%
### Лучшее значение базовой модели (LGBM в дарт режиме) - 0.9347. Скор стекинга 0.93646, что не сильно больше. Как и предполагалось, бустинги сильно коррелируют друг с другом, а ExtraTrees, KNN, SVM и MLP дают более низкий, чем у бустингов и леса, скор. При этом вносят разнообразие. Необходимо найти баланс точности и разнообразия базовых моделей. 
### Лучший скор дает комбинация: Catboost, Lgbm_gbdt, Lgbm_goss, Random_forest, SVM_poly

In [66]:
estimators = [
    ('Catboost', CatBoostClassifier(
                                    **best_catboost_params, 
                                    **invariant_catboost_params,
                                    )
    ),
    
    ('Lgbm_gbdt', LGBMClassifier(random_state=15,
                                **best_lgbm_params, 
                                **invariant_lgbm_params
                                )
    ),

    
    ('Lgbm_goss', LGBMClassifier(random_state=17,
                                **best_lgbm_goss_params, 
                                **invariant_lgbm_params
                                )
    ),

    
    ('Random_forest', make_pipeline(preprocessor, RandomForestClassifier(
                                            **best_randomforest_params, 
                                            **invariant_forest_params
                                            )
                                   )
    ),
    
    
    ("SVM_poly", make_pipeline(preprocessor, SVC(random_state=21, 
                                                 kernel='poly', 
                                                 **best_svm_poly_params))),

]

stacking_classifier = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(verbose=False, random_state=24,),
    n_jobs=-1,
    verbose=False,
)

stacking_classifier

StackingClassifier(estimators=[('Catboost',
                                <catboost.core.CatBoostClassifier object at 0x0000014745F5D750>),
                               ('Lgbm_gbdt',
                                LGBMClassifier(bagging_fraction=0.9891316013437009,
                                               bagging_freq=6,
                                               cat_feature=[0, 1, 2],
                                               colsample_bytree=0.974895774239529,
                                               extra_trees='False',
                                               learning_rate=0.27938850966497325,
                                               min_data_in_leaf=1,
                                               min_gain_to_split=0, num_class=9,
                                               num_le...
                                                                                  Pipeline(steps=[('imputer',
                                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                                  ('onehot',
                                                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                                                  ['model',
                                                                                   'car_type',
                                                                                   'fuel_type'])])),
                                                ('svc',
                                                 SVC(C=1.3054435718959585,
                                                     coef0=0.4492718754629773,
                                                     degree=2, gamma='auto',
                                                     kernel='poly',
                                                     random_state=21))]))],
                   final_estimator=LogisticRegression(random_state=24,
                                                      verbose=False),
                   n_jobs=-1, verbose=False)

In [67]:
stacking_classifier.fit(X_train, y_train)

StackingClassifier(estimators=[('Catboost',
                                <catboost.core.CatBoostClassifier object at 0x0000014745F5D750>),
                               ('Lgbm_gbdt',
                                LGBMClassifier(bagging_fraction=0.9891316013437009,
                                               bagging_freq=6,
                                               cat_feature=[0, 1, 2],
                                               colsample_bytree=0.974895774239529,
                                               extra_trees='False',
                                               learning_rate=0.27938850966497325,
                                               min_data_in_leaf=1,
                                               min_gain_to_split=0, num_class=9,
                                               num_le...
                                                                                  Pipeline(steps=[('imputer',
                                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                                  ('onehot',
                                                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                                                  ['model',
                                                                                   'car_type',
                                                                                   'fuel_type'])])),
                                                ('svc',
                                                 SVC(C=1.3054435718959585,
                                                     coef0=0.4492718754629773,
                                                     degree=2, gamma='auto',
                                                     kernel='poly',
                                                     random_state=21))]))],
                   final_estimator=LogisticRegression(random_state=24,
                                                      verbose=False),
                   n_jobs=-1, verbose=False)

In [68]:
print(f'Recall стекинга из {len(estimators)} моделей:', recall_score(stacking_classifier.predict(X_test), y_test, average='macro'))

Recall стекинга из 5 моделей: 0.9417750084613447


### Прирост скора при стекинге относительно лучшей базовой модели достаточно скромный: 0.71 %. Что, вероятно, связано с высоким скором базовых моделей и недостатком данных. Разнообразные комбинации бустингов, деревьев, KNN, SVM и MLP в перспективе должны давать хорошее разнообразие моделей. Гиперпараметры так же подбирались с целью обеспечить несмещенную оценку на кросс-валидации и возможность добавить новую информацию. Использование логистической регрессии, что по сути делает из стекинга взвешенный блендинг, предотвращает переобучение мета-модели.
### Изучение данных, feature  engineering и feature selection зачастую, как и в случае данной задачи, сильнее повышает скор, чем подбор гиперпараметров и использование разных техник по типу стекинга